In [1]:
import sys
sys.path.append('../src/')

In [2]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import mrcutils
#help(mrcutils)

### 1. split mrcs in set of mrc files

In [3]:
mrcs_dir='/Users/fpoitevi/gdrive/cryoEM/Projects/2016-rib70s-F20/data/RawData/Nov10-2016/'
mrcskey ='70S_apo_11102016_stack0001_movie'
mrcsname=mrcskey+'.mrcs'
mrcsfile=mrcs_dir+mrcsname

In [ ]:
! source activate eman-env; `which e2proc2d.py` $mrcsfile $mrcsname --unstacking ; conda deactivate

In [ ]:
! for mrcs in *.mrcs; do mv $mrcs tmp/${mrcs%.*}.mrc ; done

In [ ]:
#! ls *

### 2. load in array

In [4]:
nframe=50
for iframe in np.arange(nframe):
    mrcfile='./tmp/'+mrcskey+'-'+str(iframe+1).zfill(2)+'.mrc'
    #print(mrcfile)
    if iframe == 0:
        frame = mrcutils.mrc2array(mrcfile)
    else:
        frame = np.append(frame, mrcutils.mrc2array(mrcfile), axis=0)
data = frame.reshape(nframe,int(frame.shape[0]/nframe),frame.shape[1])

Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size:  (3838, 3710)
Image size: 

In [5]:
data = data[:,0:np.minimum(data.shape[1],data.shape[2]),0:np.minimum(data.shape[1],data.shape[2])]

In [6]:
data.shape

(50, 3710, 3710)

### 3. Now for some fun

In [7]:
data_min  = np.amin(data,axis=0)
data_mean = np.mean(data,axis=0)
data_std  = np.std(data,axis=0)
data_max  = np.amax(data,axis=0)

In [8]:
list_data = (data_min,data_mean,data_std,data_max)
list_mrc  = ('tmp/min.mrc','tmp/mean.mrc','tmp/std.mrc','tmp/max.mrc')

In [9]:
for dat, mrc in zip(list_data, list_mrc):
    mrcutils.create_mrc(mrcfile_out=mrc,signal_data=dat,noise_data=dat)
#fig = plt.figure(figsize=(12,12),dpi=360)
#plt.imshow(data_max)
#plt.colorbar()

... about to write  tmp/min.mrc
... about to write  tmp/mean.mrc
... about to write  tmp/std.mrc
... about to write  tmp/max.mrc


In [10]:
data_slice = np.zeros((3,data.shape[1],data.shape[2]))
for i in np.arange(3):
    data_slice[i,...] = np.mean(data[i:48:3,...],axis=0)
data_slice.shape

(3, 3710, 3710)

In [11]:
data_slice_min = np.amin(data_slice,axis=0)

In [12]:
mrcutils.create_mrc(mrcfile_out='./tmp/slice.mrc',signal_data=data_slice_min,noise_data=data_slice_min)

... about to write  ./tmp/slice.mrc


In [13]:
diff = data_mean - data_slice_min

In [14]:
mrcutils.create_mrc(mrcfile_out='./tmp/diff.mrc',signal_data=diff,noise_data=diff)

... about to write  ./tmp/diff.mrc


In [15]:
diff_psd = mrcutils.psd(diff)

In [16]:
mrcutils.create_mrc(mrcfile_out='./tmp/diff_psd.mrc',signal_data=diff_psd,noise_data=diff_psd)

... about to write  ./tmp/diff_psd.mrc


In [17]:
data_slice_min_psd = mrcutils.psd(data_slice_min)

In [18]:
mrcutils.create_mrc(mrcfile_out='./tmp/slice_psd.mrc',signal_data=data_slice_min_psd,noise_data=data_slice_min_psd)

... about to write  ./tmp/slice_psd.mrc


In [19]:
data_mean_psd = mrcutils.psd(data_mean)

In [20]:
mrcutils.create_mrc(mrcfile_out='./tmp/mean_psd.mrc',signal_data=data_mean_psd,noise_data=data_mean_psd)

... about to write  ./tmp/mean_psd.mrc


### prepare CTF

In [21]:
keyword='diff'
ctf_input = './tmp/'+keyword+'.in'
mrcin     = './'+keyword+'.mrc'
ctffile   = './'+keyword+'_ctf.mrc'
mrcutils.prepare_ctffind(ctf_input,mrcin=mrcin,mrcout=ctffile)